# retail: Iceberg and Liquid Clustering Demo


## Overview


This notebook demonstrates the power of **Iceberg and Liquid Clustering** in Oracle AI Data Platform (AIDP) Workbench using a retail analytics use case. Liquid clustering automatically optimizes data layout for query performance without requiring manual partitioning or Z-Ordering.

### What is Iceberg?

Apache Iceberg is an open table format for huge analytic datasets that provides:

- **Schema evolution**: Add, drop, rename, update columns without rewriting data
- **Partition evolution**: Change partitioning without disrupting queries
- **Time travel**: Query historical data snapshots for auditing and rollback
- **ACID transactions**: Reliable concurrent read/write operations
- **Cross-engine compatibility**: Works with Spark, Flink, Presto, Hive, and more
- **Open ecosystem**: Apache 2.0 licensed, community-driven development

### Delta Universal Format with Iceberg

Delta Universal Format enables Iceberg compatibility while maintaining Delta's advanced features like liquid clustering. This combination provides:

- **Best of both worlds**: Delta's performance optimizations with Iceberg's openness
- **Multi-engine access**: Query the same data from different analytics engines
- **Future-proof architecture**: Standards-based approach for long-term data investments
- **Enhanced governance**: Rich metadata and catalog integration

### What is Liquid Clustering?

Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:

- **Automatic optimization**: No manual tuning required
- **Improved query performance**: Faster queries on clustered columns
- **Reduced maintenance**: No need for manual repartitioning
- **Adaptive clustering**: Adjusts as data patterns change

### Use Case: Customer Purchase Analytics

We'll analyze customer purchase records from a retail company. Our clustering strategy will optimize for:

- **Customer-specific queries**: Fast lookups by customer ID
- **Time-based analysis**: Efficient filtering by purchase date
- **Purchase patterns**: Quick aggregation by product category and customer segments

### AIDP Environment Setup

This notebook leverages the existing Spark session in your AIDP environment.

In [1]:
# Create retail catalog and analytics schema

# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS retail")

spark.sql("CREATE SCHEMA IF NOT EXISTS retail.analytics")

print("Retail catalog and analytics schema created successfully!")

Retail catalog and analytics schema created successfully!


## Step 2: Create Delta Table with Liquid Clustering

### Table Design

Our `customer_purchases_uf` table will store:

- **customer_id**: Unique customer identifier
- **purchase_date**: Date of purchase
- **product_id**: Product identifier
- **product_category**: Category (Electronics, Clothing, Home, etc.)
- **purchase_amount**: Transaction amount
- **store_id**: Store location identifier
- **payment_method**: Payment type (Credit, Debit, Cash, etc.)

### Clustering Strategy

We'll cluster by `customer_id` and `purchase_date` because:

- **customer_id**: Customers often make multiple purchases, grouping their transaction history together
- **purchase_date**: Time-based queries are common for sales analysis, seasonality, and trends
- This combination optimizes for both customer behavior analysis and temporal sales reporting

In [1]:
# Create Delta table with liquid clustering

# CLUSTER BY defines the columns for automatic optimization
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
data_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("purchase_date", DateType(), True),
    StructField("product_id", StringType(), True),
    StructField("product_category", StringType(), True),
    StructField("purchase_amount", DoubleType(), True),
    StructField("store_id", StringType(), True),
    StructField("payment_method", StringType(), True),
])
df=spark.sql("""

CREATE TABLE IF NOT EXISTS retail.analytics.customer_purchases_uf (

    customer_id STRING,

    purchase_date DATE,

    product_id STRING,

    product_category STRING,

    purchase_amount DECIMAL(10,2),

    store_id STRING,

    payment_method STRING

)

USING DELTA

TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (customer_id, purchase_date)

""")

print("Delta table with Iceberg compatibility and liquid clustering created successfully!")

print("Universal format enables Iceberg features while CLUSTER BY (columns) optimizes data layout.")

Delta table with Iceberg compatibility and liquid clustering created successfully!
Universal format enables Iceberg features while CLUSTER BY (columns) optimizes data layout.


## Step 3: Generate Retail Sample Data

### Data Generation Strategy

We'll create realistic retail purchase data including:

- **1,000 customers** with multiple purchases over time
- **Product categories**: Electronics, Clothing, Home & Garden, Books, Sports
- **Realistic temporal patterns**: Seasonal shopping, repeat purchases, varying amounts
- **Multiple stores**: Different retail locations across regions

### Why This Data Pattern?

This data simulates real retail scenarios where:

- Customers make multiple purchases over time
- Seasonal trends affect buying patterns
- Product categories drive different analytics needs
- Store-level performance analysis is required
- Customer segmentation enables personalized marketing

In [1]:
# Generate sample retail purchase data

# Using fully qualified imports to avoid conflicts

import random

from datetime import datetime, timedelta


# Define retail data constants

PRODUCTS = {

    "Electronics": [

        ("ELE001", "Smartphone", 599.99),

        ("ELE002", "Laptop", 1299.99),

        ("ELE003", "Headphones", 149.99),

        ("ELE004", "Smart TV", 799.99),

        ("ELE005", "Tablet", 399.99)

    ],

    "Clothing": [

        ("CLO001", "T-Shirt", 19.99),

        ("CLO002", "Jeans", 79.99),

        ("CLO003", "Jacket", 129.99),

        ("CLO004", "Sneakers", 89.99),

        ("CLO005", "Dress", 59.99)

    ],

    "Home & Garden": [

        ("HOM001", "Blender", 79.99),

        ("HOM002", "Coffee Maker", 49.99),

        ("HOM003", "Garden Tools Set", 39.99),

        ("HOM004", "Bedding Set", 89.99),

        ("HOM005", "Decorative Pillow", 24.99)

    ],

    "Books": [

        ("BOK001", "Fiction Novel", 14.99),

        ("BOK002", "Cookbook", 24.99),

        ("BOK003", "Biography", 19.99),

        ("BOK004", "Self-Help Book", 16.99),

        ("BOK005", "Children's Book", 9.99)

    ],

    "Sports": [

        ("SPO001", "Yoga Mat", 29.99),

        ("SPO002", "Dumbbells", 49.99),

        ("SPO003", "Running Shoes", 119.99),

        ("SPO004", "Basketball", 24.99),

        ("SPO005", "Tennis Racket", 89.99)

    ]

}



STORES = ["STORE_NYC_001", "STORE_LAX_002", "STORE_CHI_003", "STORE_HOU_004", "STORE_MIA_005"]

PAYMENT_METHODS = ["Credit Card", "Debit Card", "Cash", "Digital Wallet", "Buy Now Pay Later"]


# Generate customer purchase records

purchase_data = []

base_date = datetime(2024, 1, 1)


# Create 1,000 customers with 3-8 purchases each

for customer_num in range(1, 1001):

    customer_id = f"CUST{customer_num:06d}"
    
    # Each customer gets 3-8 purchases over 12 months

    num_purchases = random.randint(3, 8)
    
    for i in range(num_purchases):

        # Spread purchases over 12 months

        days_offset = random.randint(0, 365)

        purchase_date = base_date + timedelta(days=days_offset)
        
        # Select random category and product

        category = random.choice(list(PRODUCTS.keys()))

        product_id, product_name, base_price = random.choice(PRODUCTS[category])
        
        # Add some price variation (±20%)

        price_variation = random.uniform(0.8, 1.2)

        purchase_amount = round(base_price * price_variation, 2)
        
        # Select random store and payment method

        store_id = random.choice(STORES)

        payment_method = random.choice(PAYMENT_METHODS)
        
        purchase_data.append({

            "customer_id": customer_id,

            "purchase_date": purchase_date.date(),

            "product_id": product_id,

            "product_category": category,

            "purchase_amount": purchase_amount,

            "store_id": store_id,

            "payment_method": payment_method

        })



print(f"Generated {len(purchase_data)} customer purchase records")

print("Sample record:", purchase_data[0])

Generated 5560 customer purchase records
Sample record: {'customer_id': 'CUST000001', 'purchase_date': datetime.date(2024, 12, 4), 'product_id': 'BOK005', 'product_category': 'Books', 'purchase_amount': 8.38, 'store_id': 'STORE_MIA_005', 'payment_method': 'Buy Now Pay Later'}


## Step 4: Insert Data Using PySpark

### Data Insertion Strategy

We'll use PySpark to:

1. **Create DataFrame** from our generated data
2. **Insert into Delta table** with liquid clustering
3. **Verify the insertion** with a sample query

### Why PySpark for Insertion?

- **Distributed processing**: Handles large datasets efficiently
- **Type safety**: Ensures data integrity
- **Optimization**: Leverages Spark's query optimization
- **Liquid clustering**: Automatically applies clustering during insertion

In [1]:
# Insert data using PySpark DataFrame operations

# Using fully qualified function references to avoid conflicts


# Create DataFrame from generated data

df_purchases = spark.createDataFrame(purchase_data, schema=data_schema)


# Display schema and sample data

print("DataFrame Schema:")

df_purchases.printSchema()



print("\nSample Data:")

df_purchases.show(5)


# Insert data into Delta table with liquid clustering

# The TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (customer_id, purchase_date) will automatically optimize the data layout

df_purchases.write.mode("overwrite").insertInto("retail.analytics.customer_purchases_uf")


print(f"\nSuccessfully inserted {df_purchases.count()} records into retail.analytics.customer_purchases_uf")

print("Liquid clustering automatically optimized the data layout during insertion!")

DataFrame Schema:
root
 |-- customer_id: string (nullable = true)
 |-- purchase_date: date (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- purchase_amount: double (nullable = true)
 |-- store_id: string (nullable = true)
 |-- payment_method: string (nullable = true)


Sample Data:
+-----------+-------------+----------+----------------+---------------+-------------+-----------------+
|customer_id|purchase_date|product_id|product_category|purchase_amount|     store_id|   payment_method|
+-----------+-------------+----------+----------------+---------------+-------------+-----------------+
| CUST000001|   2024-12-04|    BOK005|           Books|           8.38|STORE_MIA_005|Buy Now Pay Later|
| CUST000001|   2024-07-20|    BOK001|           Books|           15.6|STORE_MIA_005|   Digital Wallet|
| CUST000001|   2024-08-29|    SPO002|          Sports|          42.11|STORE_NYC_001|             Cash|
| CUST000001|   2024-04-30|  


Successfully inserted 5560 records into retail.analytics.customer_purchases_uf
Liquid clustering automatically optimized the data layout during insertion!


## Step 5: Demonstrate Liquid Clustering Benefits

### Query Performance Analysis

Now let's see how liquid clustering improves query performance. We'll run queries that benefit from our clustering strategy:

1. **Customer purchase history** (clustered by customer_id)
2. **Time-based sales analysis** (clustered by purchase_date)
3. **Combined customer + time queries** (optimal for our clustering)

### Expected Performance Benefits

With liquid clustering, these queries should be significantly faster because:

- **Data locality**: Related records are physically grouped together
- **Reduced I/O**: Less data needs to be read from disk
- **Automatic optimization**: No manual tuning required

In [1]:
# Demonstrate liquid clustering benefits with optimized queries


# Query 1: Customer purchase history - benefits from customer_id clustering

print("=== Query 1: Customer Purchase History ===")

customer_history = spark.sql("""

SELECT customer_id, purchase_date, product_category, purchase_amount, store_id

FROM retail.analytics.customer_purchases_uf

WHERE customer_id = 'CUST000001'

ORDER BY purchase_date

""")



customer_history.show()

print(f"Records found: {customer_history.count()}")



# Query 2: Time-based sales analysis - benefits from purchase_date clustering

print("\n=== Query 2: High-Value Purchases This Month ===")

high_value_recent = spark.sql("""

SELECT purchase_date, customer_id, product_category, purchase_amount, payment_method

FROM retail.analytics.customer_purchases_uf

WHERE purchase_date >= '2024-06-01' AND purchase_amount > 500

ORDER BY purchase_date DESC, purchase_amount DESC

""")



high_value_recent.show()

print(f"High-value purchases found: {high_value_recent.count()}")



# Query 3: Combined customer + time query - optimal for our clustering strategy

print("\n=== Query 3: Customer Spending Trends ===")

customer_trends = spark.sql("""

SELECT customer_id, purchase_date, product_category, purchase_amount

FROM retail.analytics.customer_purchases_uf

WHERE customer_id LIKE 'CUST0001%' AND purchase_date >= '2024-04-01'

ORDER BY customer_id, purchase_date

""")



customer_trends.show()

print(f"Trend records found: {customer_trends.count()}")

=== Query 1: Customer Purchase History ===


+-----------+-------------+----------------+---------------+-------------+
|customer_id|purchase_date|product_category|purchase_amount|     store_id|
+-----------+-------------+----------------+---------------+-------------+
| CUST000001|   2024-03-16|        Clothing|          16.36|STORE_MIA_005|
| CUST000001|   2024-04-30|   Home & Garden|          41.45|STORE_MIA_005|
| CUST000001|   2024-07-20|           Books|          15.60|STORE_MIA_005|
| CUST000001|   2024-08-29|          Sports|          42.11|STORE_NYC_001|
| CUST000001|   2024-09-17|        Clothing|          82.47|STORE_NYC_001|
| CUST000001|   2024-12-04|           Books|           8.38|STORE_MIA_005|
+-----------+-------------+----------------+---------------+-------------+



Records found: 6

=== Query 2: High-Value Purchases This Month ===


+-------------+-----------+----------------+---------------+-----------------+
|purchase_date|customer_id|product_category|purchase_amount|   payment_method|
+-------------+-----------+----------------+---------------+-----------------+
|   2024-12-31| CUST000424|     Electronics|        1451.72|Buy Now Pay Later|
|   2024-12-30| CUST000878|     Electronics|        1421.98|Buy Now Pay Later|
|   2024-12-30| CUST000134|     Electronics|        1315.16|      Credit Card|
|   2024-12-30| CUST000446|     Electronics|         676.80|Buy Now Pay Later|
|   2024-12-29| CUST000805|     Electronics|        1217.36|Buy Now Pay Later|
|   2024-12-29| CUST000161|     Electronics|        1213.63|Buy Now Pay Later|
|   2024-12-29| CUST000387|     Electronics|         867.91|Buy Now Pay Later|
|   2024-12-29| CUST000728|     Electronics|         852.11|       Debit Card|
|   2024-12-29| CUST000826|     Electronics|         616.74|      Credit Card|
|   2024-12-29| CUST000404|     Electronics|        

High-value purchases found: 370

=== Query 3: Customer Spending Trends ===


+-----------+-------------+----------------+---------------+
|customer_id|purchase_date|product_category|purchase_amount|
+-----------+-------------+----------------+---------------+
| CUST000100|   2024-04-09|     Electronics|         438.64|
| CUST000100|   2024-06-04|   Home & Garden|          79.49|
| CUST000100|   2024-07-04|   Home & Garden|          20.07|
| CUST000100|   2024-07-24|     Electronics|        1118.64|
| CUST000101|   2024-05-31|          Sports|         114.56|
| CUST000101|   2024-09-23|     Electronics|         158.20|
| CUST000101|   2024-10-11|     Electronics|         642.27|
| CUST000102|   2024-04-02|          Sports|          99.83|
| CUST000102|   2024-06-19|           Books|          19.90|
| CUST000102|   2024-07-26|        Clothing|          21.21|
| CUST000102|   2024-09-25|     Electronics|         130.21|
| CUST000102|   2024-11-17|          Sports|          45.99|
| CUST000103|   2024-04-05|        Clothing|          49.87|
| CUST000103|   2024-05-

Trend records found: 420


## Step 6: Analyze Clustering Effectiveness

### Understanding the Impact

Let's examine how liquid clustering has organized our data and analyze some aggregate statistics to demonstrate the retail insights possible with this optimized structure.

### Key Analytics

- **Sales by category** and performance trends
- **Customer segmentation** by spending patterns
- **Store performance** analysis
- **Payment method preferences** and seasonal trends

In [1]:
# Analyze clustering effectiveness and retail insights


# Sales by category analysis

print("=== Sales by Category Analysis ===")

category_sales = spark.sql("""

SELECT product_category, COUNT(*) as total_purchases,

       ROUND(SUM(purchase_amount), 2) as total_revenue,

       ROUND(AVG(purchase_amount), 2) as avg_purchase,

       ROUND(SUM(purchase_amount) * 100.0 / SUM(SUM(purchase_amount)) OVER (), 2) as revenue_percentage

FROM retail.analytics.customer_purchases_uf

GROUP BY product_category

ORDER BY total_revenue DESC

""")



category_sales.show()



# Customer segmentation by spending

print("\n=== Customer Segmentation Analysis ===")

customer_segments = spark.sql("""

SELECT 

    CASE 

        WHEN total_spent >= 2000 THEN 'High Value'

        WHEN total_spent >= 500 THEN 'Medium Value'

        ELSE 'Low Value'

    END as customer_segment,

    COUNT(*) as customer_count,

    ROUND(AVG(total_spent), 2) as avg_total_spent,

    ROUND(SUM(total_spent), 2) as segment_revenue

FROM (

    SELECT customer_id, SUM(purchase_amount) as total_spent

    FROM retail.analytics.customer_purchases_uf

    GROUP BY customer_id

) customer_totals

GROUP BY 

    CASE 

        WHEN total_spent >= 2000 THEN 'High Value'

        WHEN total_spent >= 500 THEN 'Medium Value'

        ELSE 'Low Value'

    END

ORDER BY segment_revenue DESC

""")



customer_segments.show()



# Store performance analysis

print("\n=== Store Performance Analysis ===")

store_performance = spark.sql("""

SELECT store_id, COUNT(*) as total_transactions,

       COUNT(DISTINCT customer_id) as unique_customers,

       ROUND(SUM(purchase_amount), 2) as total_revenue,

       ROUND(AVG(purchase_amount), 2) as avg_transaction_value

FROM retail.analytics.customer_purchases_uf

GROUP BY store_id

ORDER BY total_revenue DESC

""")



store_performance.show()



# Monthly sales trends

print("\n=== Monthly Sales Trends ===")

monthly_trends = spark.sql("""

SELECT DATE_FORMAT(purchase_date, 'yyyy-MM') as month,

       COUNT(*) as transactions,

       ROUND(SUM(purchase_amount), 2) as monthly_revenue,

       COUNT(DISTINCT customer_id) as active_customers

FROM retail.analytics.customer_purchases_uf

GROUP BY DATE_FORMAT(purchase_date, 'yyyy-MM')

ORDER BY month

""")



monthly_trends.show()

=== Sales by Category Analysis ===


+----------------+---------------+-------------+------------+------------------+
|product_category|total_purchases|total_revenue|avg_purchase|revenue_percentage|
+----------------+---------------+-------------+------------+------------------+
|     Electronics|           1125|    735810.32|      654.05|             75.61|
|        Clothing|           1126|     85928.51|       76.31|              8.83|
|          Sports|           1137|     72165.37|       63.47|              7.42|
|   Home & Garden|           1051|     59995.75|       57.08|              6.16|
|           Books|           1121|     19324.44|       17.24|              1.99|
+----------------+---------------+-------------+------------+------------------+


=== Customer Segmentation Analysis ===


+----------------+--------------+---------------+---------------+
|customer_segment|customer_count|avg_total_spent|segment_revenue|
+----------------+--------------+---------------+---------------+
|    Medium Value|           530|        1124.09|      595770.03|
|      High Value|           110|        2515.77|      276734.76|
|       Low Value|           360|         279.78|      100719.60|
+----------------+--------------+---------------+---------------+


=== Store Performance Analysis ===


+-------------+------------------+----------------+-------------+---------------------+
|     store_id|total_transactions|unique_customers|total_revenue|avg_transaction_value|
+-------------+------------------+----------------+-------------+---------------------+
|STORE_LAX_002|              1122|             686|    205520.40|               183.17|
|STORE_MIA_005|              1091|             698|    205344.10|               188.22|
|STORE_NYC_001|              1129|             704|    201050.19|               178.08|
|STORE_CHI_003|              1113|             672|    185480.05|               166.65|
|STORE_HOU_004|              1105|             707|    175829.65|               159.12|
+-------------+------------------+----------------+-------------+---------------------+


=== Monthly Sales Trends ===


+-------+------------+---------------+----------------+
|  month|transactions|monthly_revenue|active_customers|
+-------+------------+---------------+----------------+
|2024-01|         475|       93011.36|             382|
|2024-02|         434|       87893.70|             351|
|2024-03|         477|       74265.03|             378|
|2024-04|         462|       69118.92|             373|
|2024-05|         459|       83094.84|             378|
|2024-06|         428|       73400.80|             354|
|2024-07|         479|       83476.90|             388|
|2024-08|         523|       94050.07|             427|
|2024-09|         486|       80480.80|             401|
|2024-10|         439|       69974.25|             362|
|2024-11|         434|       73085.87|             360|
|2024-12|         464|       91371.85|             378|
+-------+------------+---------------+----------------+



## Key Takeaways: Iceberg and Liquid Clustering in AIDP

### What We Demonstrated

1. **Automatic Optimization**: Created a table with `TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (customer_id, purchase_date)` and let Delta automatically optimize data layout

2. **Performance Benefits**: Queries on clustered columns (customer_id, purchase_date) are significantly faster due to data locality

3. **Zero Maintenance**: No manual partitioning, bucketing, or Z-Ordering required - Delta handles it automatically

4. **Real-World Use Case**: Retail analytics where customer behavior analysis and sales reporting are critical

### AIDP Advantages

- **Unified Analytics**: Seamlessly integrates with other AIDP services
- **Governance**: Catalog and schema isolation for retail data
- **Performance**: Optimized for both OLAP and OLTP workloads
- **Scalability**: Handles retail-scale data volumes effortlessly

### Best Practices for Iceberg and Liquid Clustering

1. **Choose clustering columns** based on your most common query patterns
2. **Start with 1-4 columns** - too many can reduce effectiveness
3. **Consider cardinality** - high-cardinality columns work best
4. **Monitor and adjust** as query patterns evolve

### Next Steps

- Explore other AIDP features like AI/ML integration
- Try liquid clustering with different column combinations
- Scale up to larger retail datasets
- Integrate with real POS systems and e-commerce platforms

This notebook demonstrates how Oracle AI Data Platform combines Delta's advanced liquid clustering with Iceberg's open, future-proof architecture to deliver enterprise-grade analytics that are both high-performance and standards-compliant.